In [1]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.managers.manager import Manager
from fastlane_bot.events.interface import QueryInterface
import decimal
import random
import time
from _decimal import Decimal
from dataclasses import dataclass, asdict, field
from datetime import datetime
from typing import List, Dict, Tuple, Any, Callable
from typing import Optional

from fastlane_bot.config import Config
from fastlane_bot.helpers import (
    TxSubmitHandler,
    TxSubmitHandlerBase,
    TxReceiptHandler,
    TxReceiptHandlerBase,
    TxRouteHandler,
    TxRouteHandlerBase,
    TxHelpers,
    TxHelpersBase,
    TradeInstruction,
    Univ3Calculator,
    RouteStruct,
)
from fastlane_bot.helpers.routehandler import maximize_last_trade_per_tkn #,BalancerInputTooLargeError
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T
from fastlane_bot.tools.optimizer import CPCArbOptimizer
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.modes.pairwise_multi import FindArbitrageMultiPairwise
from fastlane_bot.modes.pairwise_single import FindArbitrageSinglePairwise
from fastlane_bot.modes.triangle_multi import ArbitrageFinderTriangleMulti
from fastlane_bot.modes.triangle_single import ArbitrageFinderTriangleSingle
from fastlane_bot.modes.triangle_single_bancor3 import ArbitrageFinderTriangleSingleBancor3
from fastlane_bot.modes.triangle_bancor_v3_two_hop import ArbitrageFinderTriangleBancor3TwoHop
from fastlane_bot.utils import num_format, log_format, num_format_float
from joblib import Parallel, delayed
from typing import List, Dict
import pytest
import math
import json
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
from fastlane_bot.modes import triangle_single_bancor3
#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v3.3.1 (05/Oct/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.2 (2023-08-27)
UniswapV3 v0.0.2 (2023-08-27)
SushiswapV2 v0.0.2 (2023-08-27)
CarbonV1 v0.0.2 (2023-08-27)
BancorV3 v0.0.2 (2023-08-27)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.2 [requirements >= 3.0 is met]


In [2]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET,) #blockchain="coinbase_base")
C.DEFAULT_MIN_PROFIT_GAS_TOKEN = 0.0001

setup_bot = CarbonBot(ConfigObj=C)
pools = None

### 
"""
Put path to log file here >>>
"""
###

path = os.path.normpath("fastlane_bot/data/tests/latest_pool_data_testing_balancer.json")
path = os.path.normpath("C:\\Users\\Kveen\\PycharmProjects\\fastlane-bot\logs\\20231122-172040\\latest_pool_data.json")

print(f"path={path}")
with open(path) as f:
    pools = json.load(f)



# with open("fastlane_bot/data/tests/latest_pool_data_testing_balancer.json") as f:
#     pools = json.load(f)

flashloan_tokens = ["BNT-FF1C","WETH-6Cc2","USDC-eB48","LINK-86CA"]
#flashloan_tokens = ["WETH-0006,USDC-2913"]

pools = [pool for pool in pools]
pools[0]
static_pools = pools
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
uniswap_v3_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v3_event_mappings.csv", low_memory=False)

tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,bancor_pol,bancor_v3"

exchanges = exchanges.split(",")


alchemy_max_block_fetch = 20
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)
static_pool_data['exchange_name'].unique()
# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    #uniswap_v3_event_mappings=uniswap_v3_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    bot.db.mgr = mgr
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot
bot = init_bot(mgr)
# add data cleanup steps from main.py
bot.db.handle_token_key_cleanup()
#bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()
tokens = bot.db.get_tokens()
ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}
#flashloan_tokens = bot.setup_flashloan_tokens(None)
#flashloan_tokens = ['WBTC-2c599', 'USDC-eB48', 'LINK-86CA', 'USDT-1ec7']


CCm = bot.setup_CCm(None)
pools = db.get_pool_data_with_tokens()


2023-11-22 18:22:36,676 [fastlane:INFO] - 
2023-11-22 18:22:36,677 [fastlane:INFO] - **********************************************
2023-11-22 18:22:36,677 [fastlane:INFO] - The logging path is set to: logs/20231122-182236\bot.log
2023-11-22 18:22:36,678 [fastlane:INFO] - **********************************************
2023-11-22 18:22:36,679 [fastlane:INFO] - 


path=C:\Users\Kveen\PycharmProjects\fastlane-bot\logs\20231122-172040\latest_pool_data.json


2023-11-22 18:22:43,365 [fastlane:INFO] - Retrieved 208 carbon pairs from contract
2023-11-22 18:22:54,298 [fastlane:INFO] - Time taken to add initial pools: 0.005980014801025391
2023-11-22 18:22:54,299 [fastlane:INFO] - Initializing the bot...
2023-11-22 18:22:54,403 [fastlane:INFO] - uniswap_v3: 2
2023-11-22 18:22:54,404 [fastlane:INFO] - sushiswap_v2: 0
2023-11-22 18:22:54,404 [fastlane:INFO] - uniswap_v2: 0
2023-11-22 18:22:54,416 [fastlane:INFO] - bancor_v2: 0
2023-11-22 18:22:54,416 [fastlane:INFO] - bancor_v3: 14
2023-11-22 18:22:54,416 [fastlane:INFO] - bancor_pol: 73
2023-11-22 18:22:54,417 [fastlane:INFO] - carbon_v1: 0
2023-11-22 18:22:54,417 [fastlane:INFO] - pancakeswap_v2: 0
2023-11-22 18:22:54,418 [fastlane:INFO] - pancakeswap_v3: 0
2023-11-22 18:22:54,418 [fastlane:INFO] - balancer: 63
2023-11-22 18:22:54,429 [fastlane:INFO] - uniswap_v3_zero_liquidity_pools: 1490
2023-11-22 18:22:54,432 [fastlane:INFO] - sushiswap_v2_zero_liquidity_pools: 88
2023-11-22 18:22:54,433 [fa

# Test Native Gas Token Wrap Unwrap [NB061]

## Test_Wrap_Unwrap_Gas_Token_In_Route_Struct

In [3]:
arb_mode = "multi_pairwise_pol"

arb_finder = bot._get_arb_finder(arb_mode)
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()

r = [arb for arb in r if len(arb[2]) >= 2]
r.sort(key=lambda x: x[0], reverse=True)



targettkn == c0.tknx = True
func = <function PairOptimizer.optimize.<locals>.<lambda> at 0x000001F39620EE60>
full_result = [-1.71098691e-11  3.37022357e-08], p_optimal=OptimizerBase.SimpleResult(result=1969.2780979618437, method='bisection', errormsg=None, context_dct=None), p_optimal_t=(0.0005078002954661287,) <class 'tuple'>
trade_instructions_dic = ({'cid': '0x57a875c1f78317b8aaa4bdf4d13742c7b4ffbc486c29ab556ba8501095f1b3a2', 'tknin': 'USDC-eB48', 'amtin': 3.370223566889763e-08, 'tknout': 'WETH-6Cc2', 'amtout': -1.7109869077103212e-11, 'error': None},)
trade_instructions = (CPCArbOptimizer.TradeInstruction(cid='0x57a875c1f78317b8aaa4bdf4d13742c7b4ffbc486c29ab556ba8501095f1b3a2', tknin='USDC-eB48', amtin=3.370223566889763e-08, tknout='WETH-6Cc2', amtout=-1.7109869077103212e-11, error=None),)

targettkn == c0.tknx = True
func = <function PairOptimizer.optimize.<locals>.<lambda> at 0x000001F471677370>
full_result = [ 2.56505928e-12 -5.10772225e-09], p_optimal=OptimizerBase.SimpleResult

In [4]:
for arb in r:

    (
        best_profit,
        best_trade_instructions_df,
        best_trade_instructions_dic,
        best_src_token,
        best_trade_instructions,
    ) = arb

    # Order the trade instructions
    (
        ordered_trade_instructions_dct,
        tx_in_count,
    ) = bot._simple_ordering_by_src_token(
        best_trade_instructions_dic, best_src_token
    )

    # Scale the trade instructions
    ordered_scaled_dcts = bot._basic_scaling(
        ordered_trade_instructions_dct, best_src_token
    )

    # Convert the trade instructions
    ordered_trade_instructions_objects = bot._convert_trade_instructions(
        ordered_scaled_dcts
    )

    # Create the tx route handler
    tx_route_handler = bot.TxRouteHandlerClass(
        trade_instructions=ordered_trade_instructions_objects
    )

    # Aggregate the carbon trades
    agg_trade_instructions = (
        tx_route_handler.aggregate_carbon_trades(ordered_trade_instructions_objects)
        if bot._carbon_in_trade_route(ordered_trade_instructions_objects)
        else ordered_trade_instructions_objects
    )

    # Calculate the trade instructions
    #try:
    calculated_trade_instructions = tx_route_handler.calculate_trade_outputs(trade_instructions=agg_trade_instructions)

    # Aggregate multiple Bancor V3 trades into a single trade
    calculated_trade_instructions = tx_route_handler.aggregate_bancor_v3_trades(
        calculated_trade_instructions
    )
    flashloan_struct = tx_route_handler.generate_flashloan_struct(
        trade_instructions_objects=calculated_trade_instructions)

    # Get the flashloan token
    fl_token = calculated_trade_instructions[0].tknin_key


    best_profit = flashloan_tkn_profit = tx_route_handler.calculate_trade_profit(
        calculated_trade_instructions
    )


    # Use helper function to calculate profit
    best_profit, flt_per_bnt, profit_usd = bot.calculate_profit(
        CCm, best_profit, fl_token
    )

    # Log the best trade instructions
    bot.handle_logging_for_trade_instructions(
        1, best_profit=best_profit  # The log id
    )

    # Use helper function to update the log dict
    log_dict = bot.update_log_dict(
        arb_mode,
        best_profit,
        profit_usd,
        flashloan_tkn_profit,
        calculated_trade_instructions,
        fl_token,
    )

    # Log the log dict
    bot.handle_logging_for_trade_instructions(2, log_dict=log_dict)  # The log id

    # Check if the best profit is greater than the minimum profit
    # if best_profit < bot.ConfigObj.DEFAULT_MIN_PROFIT:
    #     bot.ConfigObj.logger.info(
    #         f"Opportunity with profit: {num_format(best_profit)} does not meet minimum profit: {bot.ConfigObj.DEFAULT_MIN_PROFIT}, discarding."
    #     )


    # Get the flashloan amount and token address
    flashloan_amount = int(calculated_trade_instructions[0].amtin_wei)
    flashloan_token_address = bot.ConfigObj.w3.toChecksumAddress(
        bot.db.get_token(key=fl_token).address
    )

    # Log the flashloan amount and token address
    bot.handle_logging_for_trade_instructions(
        3,  # The log id
        flashloan_amount=flashloan_amount,
    )

    # Encode the trade instructions
    encoded_trade_instructions = tx_route_handler.custom_data_encoder(
        calculated_trade_instructions
    )

    # Get the deadline
    deadline = bot._get_deadline(None)

    # Get the route struct
    route_struct = [
        asdict(rs)
        for rs in tx_route_handler.get_route_structs(
            encoded_trade_instructions, deadline
        )
    ]
    route_struct = maximize_last_trade_per_tkn(route_struct=route_struct)
    # Check if the result is None

    route_struct = tx_route_handler.add_wrap_or_unwrap_trades_to_route(trade_instructions=calculated_trade_instructions, route_struct=route_struct, flashloan_struct=flashloan_struct)
    
    print(f"\nNEW ROUTE STRUCT:\nflashloan_struct={flashloan_struct}\nroute_struct={route_struct}\n\n")
    assert flashloan_struct[0]["sourceTokens"][0] in route_struct[0]["sourceToken"], f"Flashloan token doesn't match trade 0 source token, expected {flashloan_struct[0]['sourceTokens'][0]} found {route_struct[0]['sourceToken']}"
    assert flashloan_struct[0]["sourceTokens"][0] in route_struct[-1]["targetToken"], f"Flashloan token doesn't match trade -1 target token, expected {flashloan_struct[0]['sourceTokens'][0]} found {route_struct[-1]['targetToken']}"

    for idx, route in enumerate(route_struct):
        if idx == 0:
            continue
        assert route["sourceToken"] == route_struct[idx - 1]["targetToken"]


    # Get the cids
    cids = list({ti["cid"].split("-")[0] for ti in best_trade_instructions_dic})


    # Log the route_struct
    bot.handle_logging_for_trade_instructions(
        4,  # The log id
        flashloan_amount=flashloan_amount,
        flashloan_token_address=flashloan_token_address,
        route_struct=route_struct,
        best_trade_instructions_dic=best_trade_instructions_dic,
    )


2023-11-22 18:22:54,542 [fastlane:INFO] - [2023-11-22T18:22:54::1700670174] |calculated_arb| == {'type': 'multi_pairwise_pol', 'profit_gas_token': 0.0006, 'profit_usd': 1.249, 'flashloan': [{'token': 'WETH-6Cc2', 'amount': 0.0001, 'profit': 0.0006}], 'trades': [{'trade_index': 0, 'exchange': 'bancor_pol', 'tkn_in': 'WETH-6Cc2', 'amount_in': 0.0001, 'tkn_out': 'LQTY-c54D', 'amt_out': 1.0313, 'cid0': 'f49f151b91'}, {'trade_index': 1, 'exchange': 'balancer', 'tkn_in': 'LQTY-c54D', 'amount_in': 1.0313, 'tkn_out': 'WETH-6Cc2', 'amt_out': 0.0007, 'cid0': '58ed757ec9'}]}
2023-11-22 18:22:54,704 [fastlane:INFO] - [2023-11-22T18:22:54::1700670174] |calculated_arb| == {'type': 'multi_pairwise_pol', 'profit_gas_token': 0.0003, 'profit_usd': 0.5802, 'flashloan': [{'token': 'WETH-6Cc2', 'amount': 0.0, 'profit': 0.0003}], 'trades': [{'trade_index': 0, 'exchange': 'bancor_pol', 'tkn_in': 'WETH-6Cc2', 'amount_in': 0.0, 'tkn_out': 'MPH-35C5', 'amt_out': 1.3813, 'cid0': '0c0cbd0848'}, {'trade_index': 1,


NEW ROUTE STRUCT:
flashloan_struct=[{'platformId': 2, 'sourceTokens': ['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'], 'sourceAmounts': [58971023276518]}]
route_struct=[{'platformId': 8, 'sourceToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'targetToken': '0x6DEA81C8171D0bA574754EF6F8b412F2Ed88c54D', 'sourceAmount': 0, 'minTargetAmount': 1031259230909526790, 'deadline': 1701309388, 'customAddress': '0x6DEA81C8171D0bA574754EF6F8b412F2Ed88c54D', 'customInt': 0, 'customData': '0x'}, {'platformId': 7, 'sourceToken': '0x6DEA81C8171D0bA574754EF6F8b412F2Ed88c54D', 'targetToken': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'sourceAmount': 0, 'minTargetAmount': 703567455387578, 'deadline': 1701309388, 'customAddress': '0x5f98805A4E8be255a32880FDeC7F6728C6568bA0', 'customInt': 88775372928884717379118053465101264193563178548212130870892830834284751225322, 'customData': '0x'}, {'platformId': 10, 'sourceToken': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 'targetToken': '0xEeeeeEeeeEeEeeE

In [5]:
assert False

AssertionError: 

In [ ]:
flashloan_struct=[{'platformId': 2, 'sourceTokens': ['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'], 'sourceAmounts': [7000000000000]}]
route_struct=[{'platformId': 2, 'sourceToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'targetToken': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'sourceAmount': 0, 'minTargetAmount': 18635342733368736, 'deadline': 1701301714, 'customAddress': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'customInt': 0, 'customData': '0x'}, {'platformId': 8, 'sourceToken': '0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C', 'targetToken': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'sourceAmount': 0, 'minTargetAmount': 99990000000000000000, 'deadline': 1701301714, 'customAddress': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE', 'customInt': 0, 'customData': '0x'}]


In [ ]:
from web3 import Web3
tenderly_web3 = Web3(Web3.HTTPProvider(f"https://mainnet.rpc.tenderly.co/18bd6519-1501-4eae-b89f-c8db0466b10a"))

from fastlane_bot.data.abi import FAST_LANE_CONTRACT_ABI

def get_nonce(address: str) -> int:
    """
    Gets the nonce for the transaction.

    Parameters
    ----------
    address: str
        The address.

    Returns
    -------
    int
        The nonce for the transaction.
    """
    return tenderly_web3.eth.getTransactionCount(address)

tx_submit_handler = TxSubmitHandler(
            ConfigObj=bot.ConfigObj,
            route_struct=route_struct,
            flashloan_struct=flashloan_struct,
            src_address=0,
            src_amount=0,
)
tx_submit_handler.w3 = tenderly_web3

arb_contract = tenderly_web3.eth.contract(
            address=tenderly_web3.toChecksumAddress(
                bot.ConfigObj.network.FASTLANE_CONTRACT_ADDRESS
            ),
            abi=FAST_LANE_CONTRACT_ABI,
        )

tx_submit_handler.arb_contract = arb_contract


tx = tx_submit_handler.submit_transaction_tenderly(
    route_struct=route_struct, src_address=0, src_amount=0, flashloan_struct=flashloan_struct
)